# 共轭梯度方法的比较

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import time

def targetfunction(xs,length):# 目标函数,xs是一个列向量
    result=0
    for i in range(length):
        result+=(math.exp(xs[i])-xs[i])
    return result
    
def grad(xs,length):# 目标函数的梯度,xs是一个列向量
    result=np.zeros((length,1))
    for i in range(length):
        result[i]=math.exp(xs[i])-1
    return np.mat(result)
    return np.exp(xs)-1
    
def Armijo(xk,dk,length):# 非精确线搜索，采用armijo方法
    alpha=1
    beta=0.5
    sigma=0.2
    while True:
        if targetfunction(xk+alpha*dk,length)<=targetfunction(xk,length)+sigma*alpha*grad(xk,length).T*dk:
            break
        alpha*=beta
    result=xk+alpha*dk
    return result

def ConjugateMethod(x0,length,method,searchmethod,epsilon):# 共轭梯度法,x0是一个列向量
    start1=time.time()
    xk=x0
    dk=-grad(xk,length)
    gk=grad(xk,length)
    betak=np.zeros((length,1))
    k=0
    start = time.process_time()
    while True:
        if np.linalg.norm(grad(xk,length))<epsilon:
            break# 终止条件
        #根据方向进行精确线搜索
        if searchmethod=='Armijo':
            xk=Armijo(xk,dk,length)
        elif searchmethod=='Armijo':
            xk=Accurate(xk,dk,length)
        #确定方向
        gk1=gk
        dk1=dk
        gk=grad(xk,length)
        if method=='PRP':
            betak=(gk.T*(gk-gk1))/(gk1.T*gk1)
        elif method=='FR':
            betak=(gk.T*gk)/(gk1.T*gk1)
        elif method=='PRP+':
            betak=(gk.T*(gk-gk1))/(gk1.T*gk1)
            if betak<0:
                betak=0
        elif method=='CD':
            betak=-(gk.T*gk)/(dk1.T*gk1)
        elif method=='DY':
            betak=(gk.T*gk)/(dk1.T*(gk-gk1))
        k+=1
        dk=-gk+dk1*betak
    end = time.process_time() - start
    end1=time.time()-start1
    return xk,k,end,end1

n=1000
X=np.linspace(1/n,1,n)
x0=np.mat(X).T
epsilon=1e-6
print("采用非精确线搜索armijo方法：")
xk,k,end,end1=ConjugateMethod(x0,n,'PRP','Armijo',epsilon)
print(f"PRP:迭代值：{targetfunction(xk,n)},迭代次数：{k},CPU时间：{end},实际执行时间：{end1}")
xk,k,end,end1=ConjugateMethod(x0,n,'FR','Armijo',epsilon)
print(f"FR:迭代值：{targetfunction(xk,n)},迭代次数：{k},CPU时间：{end},实际执行时间：{end1}")
xk,k,end,end1=ConjugateMethod(x0,n,'PRP+','Armijo',epsilon)
print(f"PRP+:迭代值：{targetfunction(xk,n)},迭代次数：{k},CPU时间：{end},实际执行时间：{end1}")
xk,k,end,end1=ConjugateMethod(x0,n,'CD','Armijo',epsilon)
print(f"CD:迭代值：{targetfunction(xk,n)},迭代次数：{k},CPU时间：{end},实际执行时间：{end1}")
xk,k,end,end1=ConjugateMethod(x0,n,'DY','Armijo',epsilon)
print(f"DY:迭代值：{targetfunction(xk,n)},迭代次数：{k},CPU时间：{end},实际执行时间：{end1}")


采用非精确线搜索armijo方法：
PRP:迭代值：[[1000.]],迭代次数：10,CPU时间：0.546875,实际执行时间：1.9786758422851562
FR:迭代值：[[1000.]],迭代次数：14,CPU时间：0.234375,实际执行时间：0.45911502838134766
PRP+:迭代值：[[1000.]],迭代次数：6,CPU时间：0.3125,实际执行时间：1.2204349040985107
CD:迭代值：[[1000.]],迭代次数：35,CPU时间：1.4375,实际执行时间：4.51496958732605
DY:迭代值：[[1000.]],迭代次数：19,CPU时间：0.828125,实际执行时间：2.1485743522644043


由此可以得出，方法的优劣性PRP+>PRP>FR>DY>CD。与小组其他成员的结果有差异。

# 差异原因：
目标函数的定义中使用了循环。增加了代码复杂度，拖慢了执行时间

armijo方法的参数设置会导致迭代的次数不同

电脑不同配置的性能不同，运行时间有差异


# 改进后：

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import time

def targetfunction(xs,length):# 目标函数,xs是一个列向量
    '''result=0
    for i in range(length):
        result+=(math.exp(xs[i])-xs[i])
    return result'''
    return np.sum(np.exp(xs)-xs)

def grad(xs,length):# 目标函数的梯度,xs是一个列向量
    '''result=np.zeros((length,1))
    for i in range(length):
        result[i]=math.exp(xs[i])-1
    return np.mat(result)'''
    return np.exp(xs)-1
    
def Armijo(xk,dk,length):# 非精确线搜索，采用armijo方法
    alpha=1
    beta=0.8
    sigma=0.5
    while True:
        if targetfunction(xk+alpha*dk,length)<=targetfunction(xk,length)+sigma*alpha*grad(xk,length).T*dk:
            break
        alpha*=beta
    result=xk+alpha*dk
    return result

def ConjugateMethod(x0,length,method,searchmethod,epsilon):# 共轭梯度法,x0是一个列向量
    start1=time.time()
    xk=x0
    dk=-grad(xk,length)
    gk=grad(xk,length)
    betak=np.zeros((length,1))
    k=0
    start = time.process_time()
    while True:
        if np.linalg.norm(grad(xk,length))<epsilon:
            break# 终止条件
        #根据方向进行精确线搜索
        if searchmethod=='Armijo':
            xk=Armijo(xk,dk,length)
        elif searchmethod=='Armijo':
            xk=Accurate(xk,dk,length)
        #确定方向
        gk1=gk
        dk1=dk
        gk=grad(xk,length)
        if method=='PRP':
            betak=(gk.T*(gk-gk1))/(gk1.T*gk1)
        elif method=='FR':
            betak=(gk.T*gk)/(gk1.T*gk1)
        elif method=='PRP+':
            betak=(gk.T*(gk-gk1))/(gk1.T*gk1)
            if betak<0:
                betak=0
        elif method=='CD':
            betak=-(gk.T*gk)/(dk1.T*gk1)
        elif method=='DY':
            betak=(gk.T*gk)/(dk1.T*(gk-gk1))
        k+=1
        dk=-gk+dk1*betak
    end = time.process_time() - start
    end1=time.time()-start1
    return xk,k,end,end1

n=1000
X=np.linspace(1/n,1,n)
x0=np.mat(X).T
epsilon=1e-6
print("采用非精确线搜索armijo方法：")
xk,k,end,end1=ConjugateMethod(x0,n,'PRP','Armijo',epsilon)
print(f"PRP:迭代值：{targetfunction(xk,n)},迭代次数：{k},CPU时间：{end},实际执行时间：{end1}")
xk,k,end,end1=ConjugateMethod(x0,n,'FR','Armijo',epsilon)
print(f"FR:迭代值：{targetfunction(xk,n)},迭代次数：{k},CPU时间：{end},实际执行时间：{end1}")
xk,k,end,end1=ConjugateMethod(x0,n,'PRP+','Armijo',epsilon)
print(f"PRP+:迭代值：{targetfunction(xk,n)},迭代次数：{k},CPU时间：{end},实际执行时间：{end1}")
xk,k,end,end1=ConjugateMethod(x0,n,'CD','Armijo',epsilon)
print(f"CD:迭代值：{targetfunction(xk,n)},迭代次数：{k},CPU时间：{end},实际执行时间：{end1}")
xk,k,end,end1=ConjugateMethod(x0,n,'DY','Armijo',epsilon)
print(f"DY:迭代值：{targetfunction(xk,n)},迭代次数：{k},CPU时间：{end},实际执行时间：{end1}")


采用非精确线搜索armijo方法：
PRP:迭代值：1000.0,迭代次数：10,CPU时间：0.0,实际执行时间：0.007239580154418945
FR:迭代值：1000.0,迭代次数：9,CPU时间：0.0,实际执行时间：0.00307464599609375
PRP+:迭代值：1000.0,迭代次数：10,CPU时间：0.0,实际执行时间：0.002045154571533203
CD:迭代值：1000.0,迭代次数：9,CPU时间：0.0,实际执行时间：0.0029954910278320312
DY:迭代值：1000.0,迭代次数：9,CPU时间：0.0,实际执行时间：0.0020046234130859375


可知，代码优化之后的执行时间和迭代次数有了明显提升。效果更好。
由此可以比较：代码的优劣性DY>CD>FR>PRP+>PRP。